###  Euro spot yield curve (Government) further analysis

In [1]:
# Data preparation
import datetime as dt
import numpy as np
import pandas as pd
import ipywidgets as widgets
from sklearn.preprocessing import StandardScaler



/tmp/ipykernel_5826/981370666.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import yieldcurves as yc

In [3]:
# Custom libraries
import principalcomponents as pc
import autoregressive as ar

In [4]:
# Visualization
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from matplotlib.pyplot import cm
from matplotlib.animation import FuncAnimation
from matplotlib.ticker import FormatStrFormatter

import seaborn as sns
sns.set()

# Date formats
days_loc   = mdates.DayLocator(interval = 7)
months_loc = mdates.MonthLocator()
years_loc  = mdates.YearLocator()

days_fmt   = mdates.DateFormatter("%Y-%m-%d")
months_fmt = mdates.DateFormatter("%Y-%m")
years_fmt  = mdates.DateFormatter('%Y')

In [5]:
# Autocorrelation & Stationarity
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [6]:
def rainbow(categories):
    """
    This function generates a dictionary of color codes for each category.
    """
    c_scale = cm.rainbow(np.linspace(0,1,len(categories)))
    c_dict = {}

    for i,c in zip(categories,c_scale):
        c_dict[i] = c
        
    return c_dict

In [7]:
def underline(string):
    chr_start = "\033[4m"
    chr_end = "\033[0m"
    print(chr_start + string + chr_end)

In [8]:
# Choose the maturities that should be used for calculation
maturities = [0.25, 0.5, 1, 2, 3 , 4, 5, 6, 7, 8, 9, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30]

# Choose the maturities that should be depicted in charts
maturities_visual = [0.25, 0.5, 2, 5, 10, 14, 20, 26, 30]

In [9]:
# Parameters for data preparation 
widg_start = widgets.DatePicker(description = "Start date", value = dt.date(2005,1,4), disabled = False)
widg_end   = widgets.DatePicker(description = "End date",   value = dt.date(2021,1,1), disabled = False)
widg_freq  = widgets.Dropdown(options = ["day", "week", "month"], 
                              value = "day", 
                              description = "Frequency:",
                              disabled = False)

In [10]:
# Widget for start date of testing window
widg_test = widgets.DatePicker(description = "Test start", value = dt.date(2018,1,1), disabled = False)

# Widget if time series shall be differenced
widg_diff = widgets.Dropdown(options=[True,False], value=True, description="Differencing:")

# Widget for number of principal components
widg_k = widgets.IntSlider(value=3, min=3, max=20, step=1,
                           description = "# PC:",
                           orientation = "horizontal",
                           readout = True)

# Widget for number of lags in model
widg_lag = widgets.IntSlider(value=5, min=1, max=50, step=1,
                             description="# Lags:",
                             orientation="horizontal",
                             readout=True)

# Widget for number of forecast steps
widg_steps = widgets.IntSlider(value=100, min=1, max=500, step=1,
                               description = "# Steps:",
                               orientation = "horizontal",
                               readout = True)

In [11]:
underline ("Data Parameters:")
display(widg_start)
display(widg_end)
display(widg_freq)

print ("\n")

underline ("Model Parameters")
display (widg_test)
display (widg_diff)
display (widg_k)
display (widg_lag)
display (widg_steps)

Data Parameters:


DatePicker(value=datetime.date(2005, 1, 4), description='Start date', step=1)

DatePicker(value=datetime.date(2021, 1, 1), description='End date', step=1)

Dropdown(description='Frequency:', options=('day', 'week', 'month'), value='day')



Model Parameters


DatePicker(value=datetime.date(2018, 1, 1), description='Test start', step=1)

Dropdown(description='Differencing:', options=(True, False), value=True)

IntSlider(value=3, description='# PC:', max=20, min=3)

IntSlider(value=5, description='# Lags:', max=50, min=1)

IntSlider(value=100, description='# Steps:', max=500, min=1)

In [12]:
# Store data prep parameters
start_date = widg_start.value.strftime("%Y-%m-%d")
end_date   = widg_end.value.strftime("%Y-%m-%d")
frequency  = widg_freq.value

# Store model parameters
test_start = widg_test.value.strftime("%Y-%m-%d")
diff  = widg_diff.value
k     = widg_k.value
lag   = widg_lag.value
steps = widg_steps.value

In [13]:
import pandas as pd

raw_rates= pd.read_csv('data (1).csv')



In [14]:
# Filter and clean data
clean_rates = yc.clean_rates(df    = raw_rates,
                             start = start_date, 
                             end   = end_date, 
                             freq  = frequency, 
                             maturities = maturities)

ValueError: Columns must be same length as key